<a href="https://colab.research.google.com/github/gcappare/datasharing/blob/master/AG1/Gabriel_Capparelli_AG1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 1 - AG1

Gabriel Capparelli

2019-08-10

https://colab.research.google.com/drive/1iC-IyVqsWZtclLvtEswt7AmPTSdRkhS0

https://github.com/gcappare/03MAIR---Algoritmos-de-Optimizacion---2019

En esta actividad actividad guiada se vieron ejemplos prácticos de los siguientes algoritmos:

* Divide y vencerás - QuickSort
* Voraz - Cálculo de devolución de monedas
* Vuelta Atrás - Problema de las N-Reinas

Y se contempla una práctica individual de algoritmo para encontrar los dos puntos mas cercanos


## Decorador para medir tiempo de ejecución

En clase se vió la utilidad de usar un decorador en python para medir el tiempo de ejecución de una función. En particular se explicó con la versión presentada por el compañero Ricardo Lebrón Aguilar. 

Por mi parte hice una pequeña modificación que incluye otros tres métodos de medir el tiempo ya que al investigar, el básado en time() no es el mas preciso que ofrece python y se recomienda el uso del método **perf_counter()**.


Una descripción de los métodos incluidos (https://docs.python.org/3/library/time.html):

* **perf_counter**: es el reloj con la mayor resolución posible
* **process_time**: suma del tiempo de CPU y del proceso actual del usuario (métodos como **sleep** no lo afectan)
* **monotonic**: garantiza que una llamada posterior a la función obtendrá un valor superior que una anterior aunque se cambie la hora del sistema









In [0]:
from functools import wraps
from time import time, monotonic, perf_counter, process_time, sleep
import random

# decorador Ricardo Lebrón Aguilar
def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()
        salida = f(*args, **kwargs)
        t_final = time()
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro
  
def calcular_tiempos(f):
    """Mi versión calcula tiempo de ejecución usando diversos metodos del módulo de time"""
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()
        t_inicial2 = perf_counter()
        t_inicial3 = process_time()
        t_inicial4 = monotonic()
        salida = f(*args, **kwargs)
        t_final = time() - t_inicial
        t_final2 = perf_counter() - t_inicial2
        t_final3 = process_time() - t_inicial3
        t_final4 = monotonic() - t_inicial4
        print(f"Tiempo de ejecución de la función {f.__name__!r} \ntime: {t_final:.5f}seg. \nperf_counter: {t_final2:.5f}seg. \nprocess_time: {t_final3:.5f}seg. \nmonotonic: {t_final4:.5f}seg.")
        return salida
    return cronometro

In [9]:
@calcular_tiempos
def duerme():
    print ("A dormir")
    sleep(3)
    print ("Despierto")
  

duerme()

A dormir
Despierto
Tiempo de ejecución de la función 'duerme' 
time: 3.00592seg. 
perf_counter: 3.00592seg. 
process_time: 0.00355seg. 
monotonic: 3.00592seg.


## Algoritmo Divide y Vencerás (DYV)

La idea básica de este algoritmo es dividir el problema original en dos o mas problemas mas fáciles de resolver y combinar los resultados para la solución final. En otras palabras consta de tres pasos:

1. **Divide**: Descomponer el problema y sub-problemas del mismo tipo.
2. **Vencerás**: Resolver estos problemas recursivamente.
3. Combinar las respuestas para obtener la solución final.

ref.: https://www.geeksforgeeks.org/divide-and-conquer/

Uno de los problemas clásicos que se resuelven con este algoritmo es el del ordenamiento. Aquí veremos como se implementa en el algoritmo de **QuickSort** el cuál es uno de los algoritmos mas eficientes para ordenar, que funciona dividiendo una lista en dos y luego resolviendo cada lista recursivamente, una de las claves de este algoritmo es escoger el mejor punto para dividir la lista, este punto se conoce como pivote.

En el ejemplo visto en clase el pivote se calculó sacando la media de tres valores de la lista, este parece ser el método de **"tres bandas"** que consiste en precisamente sacar la media de tres número aleatorios (como los números fueron generados aleatoriamente podemos suponer que los tres primeros son aleatorios) que estadísticamente nos dará un buen pivote. 

ref.: https://www.genbeta.com/desarrollo/implementando-el-algoritmo-quicksort

...
  
 La actividad siguió los siguientes pasos:

1. Primero probamos el caso donde la lista tiene un solo elemento, en este caso devolvemos el mismo elemento (1*)
2. Luego probamos cuando contiene dos elementos. Aquí devolvemos los dos elementos pero ordenados de menor a mayor(2*).
3. Luego  estimamos el pivote (3*)
4. Completamos el algoritmo
5. Probamos con una lista de 19 elementos (4*)
6. Luego probamos con una lista de 300 y 600 números aleatorios entre 1 y 10.000 (5*)


Se pudo observar que el uso de estos algoritmos recursivos tienen el problema que pueden llegar a consumir bastantes recursos, el ejemplo visto fue que al tratar de ordenar una lista con mas de 600 números se llegó al límite de recursividad de la herramienta usada.

In [82]:
# primero hicimos una prueba con una lista pequeña (4*)
A = [9187, 244, 4054, 9222, 8373, 4993, 5265, 5470, 4519, 7182, 2035, 3506, 4337, 7580, 2554, 2824, 8357, 4447, 7379]

# luego usamos una lista mas grande compuesta de números aleatorios. (5*)
# se pudo observar que al intentar una lista aproximadamente mayor a 600 
# el sistema arroja un error por sobrepasar límites de recursividad.
A = list(map(lambda x:random.randrange(1,10000), range(1,600)))
print('Mostramos los primeros 15 elementos para ver que están desordenados:')
print(A[:15])

def quick_sort(A):
    # si la lista tiene un elemento retornar la misma lista (1*)
    if len(A) <= 1:
        return A
    
    # si tiene dos elementos devolverlos ordenados (2*)
    if len(A) == 2:
        return [min(A), max(A)]
      
    # Calculamos el pivote (3*)
    pivote = (A[0] + A[1] + A[2]) / 3
    
    # Inicializamos las listas IZQ y DER para los valores menores al pivote (IZQ) y mayores (DER)
    IZQ = []
    DER = []
    # Recorremos la lista, si el número es menor al pivote agregamos a la lista IZQ
    # de lo contrario lo agregamos a la lista DER
    for n in A:
      if n < pivote:
        IZQ.append(n)
      else:
        DER.append(n)
        
    # Aqui se hace la recurrencia haciendo quicksort de cada una de las listas
    return quick_sort(IZQ) + quick_sort(DER)
      
    
@calcular_tiempo
def ordenar(A):
    return(quick_sort(A))
    
print('\nAhora ordenados:')
ordenar(A)[:15]

Mostramos los primeros 15 elementos para ver que están desordenados:
[1956, 6493, 4271, 453, 1030, 3920, 6362, 1123, 317, 5665, 3494, 9303, 4801, 2961, 2201]

Ahora ordenados:
Tiempo transcurrido (en segundos): 0.0018546581268310547


[76, 88, 100, 111, 127, 132, 150, 164, 168, 175, 191, 215, 220, 236, 239]

## QuickSort - Mejoras

La clave de la eficiencia de este algoritmo está en escoger el punto pivote que separa una lista en dos listas iguales. Dependiendo del pivote el algoritmo puede tener una complejidad de entre:

$$O(n \cdot log (n))\text{ y }O(n^2)$$ 

En el siguiente ejemplo puse un contador global y locales para contar las veces que se pasa por el algoritmo y asi ver las mejoras si las hay, ya que la diferencia en tiempos es tan pequeña que es muy dificil determinar si se deben al algoritmo o a algun otro proceso del cpu.

Con **pivote = (A[0] + A[1] + A[2]) / 3** obtuve ejemplos como este para 50 números, con este pivote las listas no eran muy similares, un ejemplo 31 y 16:
```
12345678901234567890123456789012345678901234567890( 50 )

1234567890123456789012345678901( 31 )

1234567890123456( 16 )
```
Con **pivote = (sum(A)/len(A))** obtuve una división mucho mas balanceada:
```
12345678901234567890123456789012345678901234567890( 50 )

1234567890123456789012345678( 28 )

1234567890123456789012( 22 )
```
Con 300 números calculé una **suma_global** de llamadas al algoritmo. Hice cinco sorteos por método de pivote dando unos promedios de: 

**pivote = (A[0] + A[1] + A[2]) / 3** => 2413 (0.001077890396118164 seg.)

**pivote = (sum(A)/len(A))** => 2272 (0.00104522705078125 seg.)

Se ve una clara mejora tanto en ciclos como en tiempo, el problema que veo es que al hacer el cálculo del promedio hay que sumar todos los números de la lista cada vez que se calcula un pivote y estoy usando una función para sumar (sum) cuando deberia usar un loop sumando todos los elementos. Por lo tanto no estoy seguro si en terminos de complejidad es mejor o peor algoritmo, pero si da un mejor pivote.

En https://en.wikipedia.org/wiki/Quicksort podemos ver que existen otros métodos como la mediana de tres o usando múltiples pivotes. Seria un ejercicio interesante probar todos a ver que diferencias hay entre cada uno.

In [120]:
# Probamos con una prueba de 100 números a ver con que método se escoge el mejor pivote
#
A = list(map(lambda x:random.randrange(1,10000), range(1,301)))

def quick_sort2(A):
    global suma_global
    # si la lista tiene un elemento retornar la misma lista (1*)
    if len(A) <= 1:
        return A
    
    # si tiene dos elementos devolverlos ordenados (2*)
    if len(A) == 2:
        return [min(A), max(A)]
      
    # Calculamos el pivote (3*)
    #pivote = (A[0] + A[1] + A[2]) / 3
    pivote = (sum(A)/len(A))
    
    # mediana de tres
    lo = 0
    hi = len(A)-1
    mid = len(A) // 2
    if A[mid] < A[lo]:
        A[mid], A[lo] = A[lo], A[mid] 
    if A[hi] < A[lo]:
        A[lo], A[hi] = A[hi], A[lo]
    if A[mid] < A[hi]:
        A[mid], A[hi] = A[hi], A[mid]
    #pivote = A[hi]
    #print(pivote)
    
    # Inicializamos las listas IZQ y DER para los valores menores al pivote (IZQ) y mayores (DER)
    IZQ = []
    DER = []
    # Recorremos la lista, si el número es menor al pivote agregamos a la lista IZQ
    # de lo contrario lo agregamos a la lista DER
    i=0
    t=0
    for n in A:
      i = 0 if i>=9 else i+1
      t += 1
      suma_global += 1
      
      #print(i, end = '')
      if n < pivote:
        IZQ.append(n)
      else:
        DER.append(n)
      
    #print('(',t,')')
    # Aqui se hace la recurrencia haciendo quicksort de cada una de las listas
    return quick_sort2(IZQ) + quick_sort2(DER)
      
    
@calcular_tiempo
def ordenar2(A):
    #pruebas para mejorar el algoritmo
    #print('Pivote:',(A[0] + A[1] + A[2]) / 3)
    #print('Media:',sum(A)/len(A))
    return(quick_sort2(A))
    

suma_global = 0
ordenar2(A)[:15]
print(suma_global)

Tiempo transcurrido (en segundos): 0.0011839866638183594
2273


## Algoritmo Voraz

Es una estrategia de búsqueda por la cual se sigue una heurística consistente en elegir la opción óptima en cada paso local con la esperanza de llegar a una solución general óptima. https://es.wikipedia.org/wiki/Algoritmo_voraz
Un problema típico que se resuelve con este algoritmo es el de la devolución de monedas.


**Problema:** Buscar las monedas para completar la cantidad con el sistema [25, 10, 5, 1]



Algoritmo para devolución de monedas usando un algoritmo voraz

In [3]:
SISTEMA= [25, 10, 5, 1]
#SISTEMA = quick_sort(SISTEMA)

@calcular_tiempo
def cambio_monedas(CANTIDAD,SISTEMA): 
    # definimos el tamaño de la lista que contiene la solución 
    # que será igual a la cantidad de elementos del sistema,
    # en este caso 4
    # e inicializamos cada elemento a 0
    SOLUCION = [0 for i in range(len(SISTEMA))]
  
    # inicializamos una variable que acumulará
    # el valor total de las monedas
    VALOR_ACUMULADO = 0 
  
    # iteramos cada valor de la lista por indice
    for i in range(len(SISTEMA)): 
        # dividimos el valor que nos va quedando entre el valor
        # del indice de la lista que vamos iterando
        # la parte entera es la cantidad de monedas
        # máximas que se puede usar para este valor
        monedas = int((CANTIDAD-VALOR_ACUMULADO)/SISTEMA[i])
        # guardamos la cantidad de monedas en la lista
        # de la solución en el indice del valor de
        # la moneda de cada iteración
        SOLUCION[i]=monedas
        # vamos sumando el total de las monedas
        # hasta el momento
        VALOR_ACUMULADO += monedas*SISTEMA[i]
    
        # si el valor de las monedas es igual a la cantidad
        # a devolver, hemos terminado y devolvemos la solucion
        if VALOR_ACUMULADO == CANTIDAD:
          return SOLUCION
  
    # si despues de recorrer todo nuestro SISTEMA la cantidad
    # acumulada no es igual a la cantidad a devolver
    # el problema no tiene solución con el sistema dado
    return "NO HAY SOLUCION"
  
print (cambio_monedas(23,SISTEMA))
  

Tiempo transcurrido (en segundos): 1.430511474609375e-05
[0, 2, 0, 3]


## Algoritmo de Cambio de moneda mejorado

Propuesta para mejorar el algoritmo presentado en clases:

1. Ordenar el SISTEMA de mayor a menor para evitar que se den fallos si el sistema no viene ordenado como es requerido.
2. Mostrar un resultado mas amigable para el usuario, indicando cuantas monedas de cada valor hay que devolver 



In [137]:
# Para ordenar la lista podriamos usar el algoritmo quick_sort
# declarado mas arriba pero tendriamos que modificarlo para
# que ordene de mayor a menor.
# En esta oportunidad usaremos el método de python list.sort
# para ordenar la lista

@calcular_tiempo
def cambio_monedas_mejorado(CANTIDAD,SISTEMA): 
    # ordenamos la cantidad de mayor a menor
    SISTEMA.sort(reverse=True)
    
    # definimos el tamaño de la lista que contiene la solución 
    # que será igual a la cantidad de elementos del sistema,
    # en este caso 4
    # e inicializamos cada elemento a 0
    SOLUCION = [0 for i in range(len(SISTEMA))]
  
    # inicializamos una variable que acumulará
    # el valor total de las monedas
    VALOR_ACUMULADO = 0 
  
    # iteramos cada valor de la lista por indice
    for i in range(len(SISTEMA)): 
        # dividimos el valor que nos va quedando entre el valor
        # del indice de la lista que vamos iterando
        # la parte entera es la cantidad de monedas
        # máximas que se puede usar para este valor
        monedas = int((CANTIDAD-VALOR_ACUMULADO)/SISTEMA[i])
        # guardamos la cantidad de monedas en la lista
        # de la solución en el indice del valor de
        # la moneda de cada iteración
        SOLUCION[i]=monedas
        # vamos sumando el total de las monedas
        # hasta el momento
        VALOR_ACUMULADO += monedas*SISTEMA[i]
    
        # si el valor de las monedas es igual a la cantidad
        # a devolver, hemos terminado y devolvemos la solucion
        if VALOR_ACUMULADO == CANTIDAD:
            print('Debe devolver el cambio de la siguiente manera:')
            for i in range(len(SISTEMA)):
              if SOLUCION[i]!=0:
                  print(SOLUCION[i],'monedas de',SISTEMA[i])
            return
  
    # si despues de recorrer todo nuestro SISTEMA la cantidad
    # acumulada no es igual a la cantidad a devolver
    # el problema no tiene solución con el sistema dado
    return "NO HAY SOLUCION"
  
SISTEMA= [5, 25, 1, 10]
cambio_monedas_mejorado(23,SISTEMA)

Debe devolver el cambio de la siguiente manera:
2 monedas de 10
3 monedas de 1
Tiempo transcurrido (en segundos): 0.0018296241760253906


## Algoritmo de vuelta atrás

**Problema:** Problema de las N-reinas

Los algoritmos de vuelta atrás se basan en recorrer el espacio completo de las soluciones posibles al problema planteado. https://es.wikibooks.org/wiki/Algoritmia/Vuelta_atr%C3%A1s



In [15]:
# Algoritmo de vuelta atras para el problema de las N-reinas
def es_prometedora(SOLUCION,etapa):
  for i in range(etapa+1):
    # verificar que no haya reinas en la misma fila
    if SOLUCION.count(SOLUCION[i]) > 1:       
      return False
  
    # verificar que no haya otra dama en diagonal
    for j in range(i+1, etapa +1 ):
      if abs(i-j) == abs(SOLUCION[i]-SOLUCION[j]) : 
        return False

  # no hay damas que amenacen
  return True


def reinas(N,solucion=[],etapa=0): 
  
  # Si es la primera llamada inicializamos
  # todos los elementos en 0
  # indicando que no hay reinas en ninguna casilla
  if len(solucion) == 0:
    solucion=[0 for i in range(N)]
      
  # iterar para cada reina/columna
  for i in range(1,N+1):
    # colocamos una reina
    solucion[etapa] = i
 
    # y revisamos si no tiene problemas
    # en esa casilla
    if es_prometedora(solucion,etapa):
      # si en la posición la reina no esta amenazada
      # y estamos en la penúltima etapa, encontramos la solución
      if etapa == N-1 :
        print('\nLa solución es:\n',solucion)
      else:
        # si no es el final seguimos llamando
        # recursivamente
        reinas(N,solucion,etapa+1)
    else:
      # no es prometedora
      None
    
    solucion[etapa] = 0
    
    
    
N = 4
@calcular_tiempos
def buscar_reinas(N):
    return(reinas(N))
    
buscar_reinas(N)


La solución es:
 [2, 4, 1, 3]

La solución es:
 [3, 1, 4, 2]
Tiempo de ejecución de la función 'buscar_reinas' 
time: 0.00081seg. 
perf_counter: 0.00081seg. 
process_time: 0.00080seg. 
monotonic: 0.00081seg.


## Mejora de visualización de la solución

Ya que no podia vizualizar bien la solución añadí código para dibujar el tablero y las reinas. Usando los caracteres de ajedrez en https://en.wikipedia.org/wiki/Chess_symbols_in_Unicode y https://www.unicode.org/charts/PDF/U2600.pdf

Funciona para 8 reinas también.

In [67]:
def reinas(N,solucion=[],etapa=0): 
  
  # Si es la primera llamada inicializamos
  # todos los elementos en 0
  # indicando que no hay reinas en ninguna casilla
  if len(solucion) == 0:
    solucion=[0 for i in range(N)]
      
  # iterar para cada reina/columna
  for i in range(1,N+1):
    # colocamos una reina
    solucion[etapa] = i
 
    # y revisamos si no tiene problemas
    # en esa casilla
    if es_prometedora(solucion,etapa):
      # si en la posición la reina no esta amenazada
      # y estamos en la penúltima etapa, encontramos la solución
      if etapa == N-1 :
        print('\nUna solución es:\n',solucion)
        # vamos pintando casilla por casilla fila por fila
        # si estamos en la casilla que corresponde con una en la lista de solución
        # pintamos una reina de lo contrario un figura de igual ancho
        # color_casilla va alternando el color de la casilla del tablero 9929/9930   9744/9746
        # lo comente proque el negro se confundia con la reina
        color_casilla = chr(9750)
        for f in range(1,N+1):
          for j in range(0,len(solucion)):
            # usamos operador ternario para seleccionar el caracter correcto.
            #color_casilla = chr(9750) if color_casilla == chr(9751) else chr(9751)
            caracter = (' '+chr(9819)+' ') if solucion[j] == f else (' '+color_casilla+' ')
            print(caracter,end='')
          # saltamos a la próxima fila
          print()
          #color_casilla = chr(9750) if color_casilla == chr(9751) else chr(9751)
      else:
        # si no es el final seguimos llamando
        # recursivamente
        reinas(N,solucion,etapa+1)
    else:
      # no es prometedora
      None
    
    solucion[etapa] = 0
    
    
N = 4
reinas(N)


Una solución es:
 [2, 4, 1, 3]
 ☖  ☖  ♛  ☖ 
 ♛  ☖  ☖  ☖ 
 ☖  ☖  ☖  ♛ 
 ☖  ♛  ☖  ☖ 

Una solución es:
 [3, 1, 4, 2]
 ☖  ♛  ☖  ☖ 
 ☖  ☖  ☖  ♛ 
 ♛  ☖  ☖  ☖ 
 ☖  ☖  ♛  ☖ 


# Práctica individual

**Problema:** Encontrar los dos puntos más cercanos

Dado un conjunto de puntos se trata de encontrar los dos puntos más cercanos

## Guía para aprendizaje:

1. Suponer en 1D, o sea, una lista de números: [3403, 4537, 9089, 9746, 7259, ….

2. Primer intento: Fuerza bruta

3.  Calcular la complejidad. ¿Se puede mejorar?

4. Segundo intento. Aplicar Divide y Vencerás

5. Calcular la complejidad. ¿Se puede mejorar?

6. Extender el algoritmo a 2D: [(1122, 6175), (135, 4076), (7296, 2741)…

7. Extender el algoritmo a 3D

In [0]:
import random
LISTA_1D = [random.randrange(1,10000) for x in range (1000)]
LISTA_2D = [(random.randrange(1,10000),random.randrange(1,10000)) for x in range (1000)]

# nota: para la prueba de 1D la LISTA_1D como esta generada arriba produce números repetidos
# incluso múltiples veces
# para hacer una mejor prueba, utilicé una lista que garantice puntos únicos y con cierta separación:
LISTA_1DU = random.sample(range(100000000), 1000)

# Lista para hacer pruebas iniciales
LISTA_PRUEBA = [100,4,76,9,32,12]

### Intento por fuerza bruta para 1D

En este algoritmo se deben hacer $ \frac{n \cdot (n-1)}{2}  $ comparaciones

De esta manera tiene una complejidad proporcional a $O(n^2)$


In [22]:
def dist_fuerza_bruta(lista):

    # inicializamos mejor_distancia con infinito
    mejor_distancia = float('inf')
    n=0
    # iterar por cada elemento en la lista
    for i in range(0,len(lista)-1):
        # comparar con los elementos que faltan
        # los que ya comparó no hace falta volver a comparar
        for j in range (i+1, len(lista)):
            n += 1
            #print('Compara',lista[i],'con',lista[j])
            distancia_puntos = abs(lista[i]-lista[j])
            if distancia_puntos < mejor_distancia:
                mejor_distancia = distancia_puntos
                puntos_mas_cercanos = (lista[i],lista[j])
    return puntos_mas_cercanos, mejor_distancia, n
        

print('\033[1m' + 'Prueba con seis puntos'+ '\033[0m')
print('Puntos mas cercanos:',dist_fuerza_bruta(LISTA_PRUEBA)[0], '\nDistancia entre puntos:', dist_fuerza_bruta(LISTA_PRUEBA)[1],'\nDemostración Complejidad:',dist_fuerza_bruta(LISTA_PRUEBA)[2])

print('\033[1m' + '\nPrueba con mil puntos'+ '\033[0m')
print('Puntos mas cercanos:',dist_fuerza_bruta(LISTA_1D)[0], '\nDistancia entre puntos:', dist_fuerza_bruta(LISTA_1D)[1],'\nDemostración Complejidad:',dist_fuerza_bruta(LISTA_1D)[2])

print('\033[1m' + '\nPrueba con mil puntos pero con una muestra aleatoria sin números repetidos'+ '\033[0m')
print('Puntos mas cercanos:',dist_fuerza_bruta(LISTA_1DU)[0], '\nDistancia entre puntos:', dist_fuerza_bruta(LISTA_1DU)[1],'\nDemostración Complejidad:',dist_fuerza_bruta(LISTA_1DU)[2])


Prueba con seis puntos
Puntos mas cercanos: (9, 12) 
Distancia entre puntos: 3 
Demostración Complejidad: 15

Prueba con mil puntos
Puntos mas cercanos: (2379, 2379) 
Distancia entre puntos: 0 
Demostración Complejidad: 499500

Prueba con mil puntos pero con una muestra aleatoria sin números repetidos
Puntos mas cercanos: (57756777, 57756707) 
Distancia entre puntos: 70 
Demostración Complejidad: 499500


## Mejora para el problema en 1 dimesión

El problema en una dimensión se puede mejorar a $O(n \cdot log (n))$ ordenando la lista y luego iterando una vez comparando los puntos adyacentes ya que puntos que no sean adyacentes estarian mas lejos.

In [27]:
def dist_mejorado(lista):
    # ordenamos la lista, esto nos supone un costo en el peor caso de O(n log (n))
    # ref. https://en.wikipedia.org/wiki/Timsort
    lista.sort()
    #print(lista)
    n = 0
    mejor_distancia = float('inf')
    # recorremos la lista una vez O(n)
    for i in range(0,len(lista)-1):
        n += 1
        #print('Compara',lista[i],'con',lista[i+1])
        distancia_puntos = abs(lista[i]-lista[i+1])
        if distancia_puntos < mejor_distancia:
            mejor_distancia = distancia_puntos
            puntos_mas_cercanos = (lista[i],lista[i+1])
    return puntos_mas_cercanos, mejor_distancia, n


print('\033[1m' + 'Aquí podemos observar una amplia mejoría con respecto a la fuerza bruta:\n'+ '\033[0m')
print('\033[1m' + 'Prueba con seis puntos'+ '\033[0m')
print('Puntos mas cercanos:',dist_mejorado(LISTA_PRUEBA)[0], '\nDistancia entre puntos:', dist_mejorado(LISTA_PRUEBA)[1],'\nDemostración Complejidad:',dist_mejorado(LISTA_PRUEBA)[2])

print('\033[1m' + '\nPrueba con mil puntos'+ '\033[0m')
print('Puntos mas cercanos:',dist_mejorado(LISTA_1D)[0], '\nDistancia entre puntos:', dist_mejorado(LISTA_1D)[1],'\nDemostración Complejidad:',dist_mejorado(LISTA_1D)[2])

print('\033[1m' + '\nPrueba con mil puntos pero con una muestra aleatoria sin números repetidos'+ '\033[0m')
print('Puntos mas cercanos:',dist_mejorado(LISTA_1DU)[0], '\nDistancia entre puntos:', dist_mejorado(LISTA_1DU)[1],'\nDemostración Complejidad:',dist_mejorado(LISTA_1DU)[2])


Aquí podemos observar una amplia mejoría con respecto a la fuerza bruta:

Prueba con seis puntos
Puntos mas cercanos: (9, 12) 
Distancia entre puntos: 3 
Demostración Complejidad: 5

Prueba con mil puntos
Puntos mas cercanos: (36, 36) 
Distancia entre puntos: 0 
Demostración Complejidad: 999

Prueba con mil puntos pero con una muestra aleatoria sin números repetidos
Puntos mas cercanos: (57756707, 57756777) 
Distancia entre puntos: 70 
Demostración Complejidad: 999


## Intento aplicando algoritmo de divide y vencerás para 1 dimensión

No creo que este algoritmo de fuerza bruta se pueda mejorar, debe utilizarse otro método. Por ejemplo con divide y venceras se puede lograr en $O(n \cdot log (n))$

https://www.youtube.com/watch?v=jAigdwcATNw



In [39]:
LISTA2D_PRUEBA =[(5, 6), (1, 1), (7, 3), (2, 4), (1, 8), (1, 5)]

Px = [item[1] for item in sorted([(pt[0],pt) for pt in LISTA2D_PRUEBA])]
Py = [item[1] for item in sorted([(pt[1],pt) for pt in LISTA2D_PRUEBA])]

def distancia_puntos(a, b):
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5


#print(distancia_puntos((0,0),(3,4)))
print(LISTA2D_PRUEBA)
print(Px)
print(Py)


[(5, 6), (1, 1), (7, 3), (2, 4), (1, 8), (1, 5)]
[(1, 1), (1, 5), (1, 8), (2, 4), (5, 6), (7, 3)]
[(1, 1), (7, 3), (2, 4), (1, 5), (5, 6), (1, 8)]


In [30]:
print([(random.randrange(1,10),random.randrange(1,10)) for x in range (6)])

[(5, 6), (1, 1), (7, 3), (2, 4), (1, 8), (1, 5)]
